In [1]:
import json
import openai

In [2]:
client = openai.Client()

In [3]:
def saudacao_por_perido(hora):
    if 5 <= 5 <12:
        return json.dumps({"saudacao": "Bom dia!"})
    elif 12 <= hora < 18:
         return json.dumps({"saudacao": "Boa tarde!"})
    elif 18 <= hora < 22:
        return json.dumps({"saudacao": "Boa noite!"})
    else:
        return json.dumps({"saudacao": "Boa madrugada!"})



In [4]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "saudacao_por_perido",
            "description": "Retorna uma saudação baseada na hora do dia",
        "parameters": {
            "type": "object",
            "properties":{
                "hora":{
                    "type": "integer",
                    "description": "A hora do dia em formato 24h"},
                },
                "required": ["hora"]
            
         }
      }
    }
]

In [5]:
funcao_disponivel = {
    "saudacao_por_perido": saudacao_por_perido
}

In [26]:
mensagens = [{"role": "user", "content": "Qual saudação o modelo me dá se for 20h?"}]

In [27]:
resposta = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=mensagens,
    tools=tools,
    tool_choice="auto"
    )

In [28]:
mensagens_resp = resposta.choices[0].message
mensagens_resp

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kT8UIWVZFKk45LKtl7TJf4fI', function=Function(arguments='{"hora":20}', name='saudacao_por_perido'), type='function')])

In [29]:
tools_calls = mensagens_resp.tool_calls
tools_calls

[ChatCompletionMessageToolCall(id='call_kT8UIWVZFKk45LKtl7TJf4fI', function=Function(arguments='{"hora":20}', name='saudacao_por_perido'), type='function')]

In [30]:
if tools_calls:
    mensagens.append(mensagens_resp)
    for tools_call in tools_calls:
        function_name = tools_call.function.name
        function_to_call = funcao_disponivel[function_name]
        function_args = json.loads(tools_call.function.arguments)
        function_response = function_to_call(
            hora= function_args.get("hora")
        )

        mensagens.append(
            {
                "tool_call_id": tools_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response

            }
        )

        segunda_resposta = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=mensagens
        )

In [31]:
mensagem_resposta = segunda_resposta.choices[0].message
mensagem_resposta

ChatCompletionMessage(content='Desculpe pelo erro, a saudação correta para as 20h é "Boa noite!".', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)